<a href="https://colab.research.google.com/github/lookinsight/ml/blob/main/20221108_ML_%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚢 로지스틱회귀: 타이타닉 생존자 예측하기

In [ ]:
import pandas as pd 
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# kaggle 
# https://www.kaggle.com/competitions/titanic
file_url = "https://raw.githubusercontent.com/bigdata-young/bigdata_16th/main/data/titanic_train.csv"
df_train = pd.read_csv(file_url)

In [ ]:
df_train.head()

In [ ]:
df_train = pd.read_csv(file_url, index_col = 0) 

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
df_train.describe(include = ["O"]) # 범주형 컬럼들 확인

In [ ]:
df_train.Embarked.unique()

In [ ]:
# 상관관계
df_train.corr()
# 0.2 ... 

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 

In [ ]:
sns.heatmap(df_train.corr()) 

In [ ]:
sns.heatmap(df_train.corr(), cmap='coolwarm') 

In [ ]:
sns.heatmap(df_train.corr(), cmap='coolwarm', vmin = -1, vmax = 1)

In [ ]:
sns.heatmap(df_train.corr(), cmap='coolwarm', vmin = -1, vmax = 1, annot = True) 

# 결측치 처리 

In [ ]:
df_train.info()

In [ ]:
df_train.Embarked.value_counts()

In [ ]:
# df_train.Embarked.unique()
df_train['Embarked'].unique()

In [ ]:
df_train['Embarked'].fillna('S').unique()

In [ ]:
df_train.Embarked = df_train['Embarked'].fillna('S') 
# df_train['Embarked'].fillna('S', inplace = True) 

In [ ]:
df_train.info()

In [ ]:
df_train.Cabin.unique()

In [ ]:
df_train.drop(columns = ['Cabin'], inplace = True) 
df_train 

In [ ]:
df_train.drop(columns = ['Ticket'], inplace = True) 

In [ ]:
df_train

In [ ]:
df_train['Name'].dtype

In [ ]:
# df_train['Title'] 
df_train.Name

In [ ]:
df_train.Name.str.extract('([A-Za-z]+)\.') # A-Z까지, a-z까지, +는 1개 이상, \. 은 여러개

In [ ]:
df_train['Title'] = df_train.Name.str.extract('([A-Za-z]+)\.')

In [ ]:
df_train.Title.value_counts()

In [ ]:
# set(df_train.Title )
df_train.Title.unique()

In [ ]:
title_unique = df_train.Title.unique()

In [ ]:
rarelist = []
for t in title_unique:
    df_tlist = list(df_train.Title).count(t)
    print(df_tlist)
    if df_tlist < 10:
        rarelist.append(t)
rarelist     # 리스트 컴프리헨션 & 삼항 연산자로 하는 법

In [ ]:
df_train.Title = df_train['Title'].replace(rarelist, "Rare")

In [ ]:
df_train.Title.value_counts()

In [ ]:
# df_train.groupby(['Title'])['Age'].mean()
title_age_mean = df_train.groupby(['Title'])['Age'].mean()

In [ ]:
for t in df_train.Title.unique():
    # df_train.loc[df_train['Age'].isnull() & (df_train.Title == 'Mr'), 'Age'] = title_age_mean['Mr'] 
    df_train.loc[df_train['Age'].isnull() & (df_train.Title == t), 'Age'] = title_age_mean[t]


# 원하는 조건의 값만 변경하는 방법

# df.loc[조건1 & 조건2, “column이름”] = “변경 값”

In [ ]:
df_train.info()

In [ ]:
df_train.drop(columns = ['Name', 'Title'], inplace = True) 

In [ ]:
df_train.info()

In [ ]:
df_train2 = pd.get_dummies(df_train, columns = ['Sex', 'Embarked'], drop_first = True)

In [ ]:
df_train2.info()

#  모델링

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X, y = (df_train2.drop(columns = ['Survived']), df_train2.Survived) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 100) 

In [ ]:
model.fit(X_train, y_train) 

In [ ]:
pred = model.predict(X_test) 

In [ ]:
#@title 모델 평가 (로지스틱 회귀 -> 분류문제) 

from sklearn.metrics import accuracy_score

accuracy_score(y_test, pred) 

In [ ]:
model.coef_

In [ ]:
pd.options.display.float_format = '{:.4f}'.format
pd.Series(model.coef_[0], index = X.columns) 

In [ ]:
def pre_processing(df : pd.DataFrame):
    df.Embarked = df.Embarked.fillna("S") 
    df.Fare = df.Fare.fillna(0)
    df['Title'] = df.Name.str.extract('([A-Za-z]+)\.')
    rarelist = [a for a in set(df['Title'])
                if list(df['Title']).count(a) < 10]
    df['Title'] = df['Title'].replace(rarelist, 'Rare') 
    title_age_mean = df.groupby(['Title'])['Age'].mean() 
    for v in df['Title'].unique():
        df.loc[df.Age.isnull() & (df.Title == v), 'Age'] = title_age_mean[v]
    df_clean = df.drop(columns=['Name', 'Ticket', 'Title', 'Cabin'])
    return pd.get_dummies(df_clean,
                          columns = ['Sex', 'Embarked'], drop_first=True)

# Kaggle Competition 제출

In [ ]:
# https://www.kaggle.com/competitions/titanic/data
file_url = "https://raw.githubusercontent.com/bigdata-young/bigdata_16th/main/data/titanic_test.csv"
# 맨 첫째줄 PassengerId를 index열로 지정
# train = pd.read_csv(f'{file_url}/titanic_train.csv', index_col=0)
submission = pd.read_csv(f'{file_url}', index_col=0)

In [ ]:
submission_df = pre_processing(submission)
submission_df

In [ ]:
submission.info()

In [ ]:
df_sub = pre_processing(submission)
df_sub

In [ ]:
pred_sub = model.predict(df_sub)
pred_sub

In [ ]:
result = pd.DataFrame({'PassengerId':df_sub.index,'Survived':pred_sub})
result

In [ ]:
# 인덱스 제외하고 저장
result.to_csv('submission.csv', index = False)